## Introduction
Investigating a reported intermittent problem with API 2.0
PoC Client is reporting that they are getting intermittent errors with the following:
1. Creating a savings account from the API
1. Changing the status to approved from API
1. Intermittently step 2 is failing

GitHub: https://github.com/MkershMambu/MambuAPINotebook/blob/master/IntermittentProblemInvestigating.ipynb

Viewer: [https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/IntermittentProblemInvestigating.ipynb](https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/IntermittentProblemInvestigating.ipynb?flush_cache=true)

In [72]:
%run API.py
%run ENV.py
from IPython.core.display import HTML
setENV(ENV)

In [73]:
%%writefile JSONBODY
{
  "overdraftSettings": {
	"allowOverdraft": true,
	"overdraftLimit": 0
  },
  "accountType": "CURRENT_ACCOUNT",
  "name": "MKCurTest1",
  "accountHolderKey": "8a8186ac692678910169288509c606bc",
  "productTypeKey": "8a8186aa69491c25016949888aeb10d2",
  "currencyCode": "EUR",
  "accountHolderType": "CLIENT"
}

Overwriting JSONBODY


In [74]:
%%writefile JSONBODY2
{
  "action": "APPROVE",
  "notes": "Approved from the API"
}

Overwriting JSONBODY2


In [77]:
import time

In [78]:
def testScenario(deleteIt=True):
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    # Step 1 -------------------- Create the account
    r = POST('{{env1}}/deposits', headers=HEADERS, params=PARAMS, body="JSONBODY")
    encodedKey = r.json()['encodedKey']
    if r.status_code != 201:
        print("Failed to create account")
    else:
        print("Created new account - {0}".format(encodedKey))
    time.sleep(2)
    # Step 2 -------------------- Change the Status to approved
    url = "{{env1}}" + "/deposits/{0}:changeState".format(encodedKey)
    r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY2")
    if r.status_code != 200:
        print("Failed to change Status of account to approved")
    else:
        print("Changed Status to approved - successful")
    time.sleep(2)
    if deleteIt == True:
        # Step 3 -------------------- Delete the account
        url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
        r = DELETE(url, headers=HEADERS, params=PARAMS)
        if r.status_code != 204:
            print("Failed to delete account")
        else:
            print("Delete account - successful")

In [79]:
for i in range(5):
    print("[{0}]".format(i), end='')
    testScenario(True)

[0]Created new account - 8a8187da6a2183a3016a27172a1723d6
Changed Status to approved - successful
Delete account - successful
[1]Created new account - 8a8186c16a21796b016a27195ff2263e
Changed Status to approved - successful
Delete account - successful
[2]Created new account - 8a8186906a21f30e016a27170fc6216e
Changed Status to approved - successful
Delete account - successful
[3]Created new account - 8a8186c16a21796b016a27195ff22642
Changed Status to approved - successful
Delete account - successful
[4]Created new account - 8a8186906a21f30e016a271977b32181
Changed Status to approved - successful
Delete account - successful


### Trying an alternative version of the scenario - testScenario2
This should be faster between API operations because there is no reading of files

This is also failing to duplicate the issue though??

In [18]:
jsonData1 = readFile("JSONBODY")
jsonData2 = readFile("JSONBODY2")

In [19]:
def testScenario2(deleteIt=True):
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    # Step 1 -------------------- Create the account
    r = POST2('{{env1}}/deposits', headers=HEADERS, params=PARAMS, data=jsonData1)
    encodedKey = r.json()['encodedKey']
    if r.status_code != 201:
        print("Failed to create account")
    else:
        print("Created new account - {0}".format(encodedKey))

    # Step 2 -------------------- Change the Status to approved
    url = "{{env1}}" + "/deposits/{0}:changeState".format(encodedKey)
    r = POST2(url, headers=HEADERS, params=PARAMS, data=jsonData2)
    if r.status_code != 200:
        print("Failed to change Status of account to approved")
    else:
        print("Changed Status to approved - successful")

    if deleteIt == True:
        # Step 3 -------------------- Delete the account
        url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
        r = DELETE(url, headers=HEADERS, params=PARAMS)
        if r.status_code != 204:
            print("Failed to delete account")
        else:
            print("Delete account - successful")

In [23]:
for i in range(5):
    print("[{0}]".format(i), end='')
    testScenario2(True)

[0]Created new account - 8a8186696a21009e016a21fb14073c09
Changed Status to approved - successful
Delete account - successful
[1]Created new account - 8a8186906a21f30e016a21fb07670030
Changed Status to approved - successful
Delete account - successful
[2]Created new account - 8a8186906a21f30e016a22056f95004c
Changed Status to approved - successful
Delete account - successful
[3]Created new account - 8a8186696a21009e016a2205693d3ce3
Changed Status to approved - successful
Delete account - successful
[4]Created new account - 8a8186c16a21796b016a220540c605f3
Changed Status to approved - successful
Delete account - successful


In [14]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = DELETE(url, headers=HEADERS, params=PARAMS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
204
Now the JSON:



In [30]:
encodedKey = "8a85879169f1de4c0169f289795519da"

In [31]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
# Parameters below set up pagination and only returns max 20
PARAMS = {'detailsLevel': 'FULL'}
url = "{{env3}}" + "/deposits/{0}".format(encodedKey)
r = GET(url, headers=HEADERS, params=PARAMS)
PRINT(r)

API status: 200
JSON Response:
{
    "addresses": [],
    "approvedDate": "2019-04-06T13:10:19+01:00",
    "assignedBranchKey": "8a85879169f1de4c0169f289795519d8",
    "clientRoleKey": "8a858ee569a54f680169bee0659c7b00",
    "creationDate": "2019-04-06T13:10:19+01:00",
    "encodedKey": "8a85879169f1de4c0169f289795519da",
    "firstName": "Jim",
    "groupLoanCycle": 0,
    "id": "443318184",
    "idDocuments": [],
    "lastModifiedDate": "2019-04-06T13:10:38+01:00",
    "lastName": "Smith",
    "loanCycle": 0,
    "notes": "",
    "preferredLanguage": "ENGLISH",
    "state": "INACTIVE"
}


In [25]:
%%writefile JSONBODY3
{
  "action": "UNDO_APPROVE",
  "notes": "Approved from the API"
}

Writing JSONBODY3


In [26]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}:changeState".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY3")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
200
Now the JSON:
{"encodedKey":"8a8186c16a21796b016a21d0cce0055f","creationDate":"2019-04-15T18:24:53+02:00","lastModifiedDate":"2019-04-15T18:29:44+02:00","id":"MGQN492","name":"MKCurTest1","accountHolderType":"CLIENT","accountHolderKey":"8a8186ac692678910169288509c606bc","accountState":"PENDING_APPROVAL","productTypeKey":"8a8186aa69491c25016949888aeb10d2","accountType":"CURRENT_ACCOUNT","currencyCode":"EUR","assignedBranchKey":"8a8186ac692678910169287cf43606af","internalControls":{},"overdraftSettings":{"allowOverdraft":true,"overdraftLimit":0E-10},"interestSettings":{"interestPaymentSettings":{"interestPaymentDates":[]}},"overdraftInterestSettings":{"interestRateSettings":{"encodedKey":"8a8186c16a21796b016a21d0cce00560","interestRate":0E-20,"interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED","interestRateSource":"FIXED_INTEREST_RATE"}},"balances":{"overdraftAmount":0E-10,"technicalOverdraftAmount":0E

## Test on the actual clients environment that is reporting the problem

In [68]:
%%writefile JSONBODY4
{
  "overdraftSettings": {
	"allowOverdraft": true,
	"overdraftLimit": 0
  },
  "accountType": "CURRENT_ACCOUNT",
  "name": "MKCurTest1",
  "accountHolderKey": "8a85879169f1de4c0169f289795519da",
  "productTypeKey": "8a8586e46a235584016a24695e6214bd",
  "currencyCode": "GBP",
  "accountHolderType": "CLIENT"
}

Overwriting JSONBODY4


In [69]:
def testScenario3(deleteIt=True):
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    # Step 1 -------------------- Create the account
    r = POST('{{env3}}/deposits', headers=HEADERS, params=PARAMS, body="JSONBODY4")
    encodedKey = r.json()['encodedKey']
    if r.status_code != 201:
        print("Failed to create account")
    else:
        print("Created new account - {0}".format(encodedKey))

    # Step 2 -------------------- Change the Status to approved
    url = "{{env3}}" + "/deposits/{0}:changeState".format(encodedKey)
    r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY2")
    if r.status_code != 200:
        print("Failed to change Status of account to approved")
    else:
        print("Changed Status to approved - successful")

    if deleteIt == True:
        # Step 3 -------------------- Delete the account
        url = "{{env3}}" + "/deposits/{0}".format(encodedKey)
        r = DELETE(url, headers=HEADERS, params=PARAMS)
        if r.status_code != 204:
            print("Failed to delete account")
        else:
            print("Delete account - successful")

In [70]:
for i in range(10):
    print("[{0}]".format(i), end='')
    testScenario3(True)

[0]Created new account - 8a8587856a25d48e016a262d1b744274
Changed Status to approved - successful
Delete account - successful
[1]Created new account - 8a85860c6a25f24b016a26825d6145c2
Changed Status to approved - successful
Delete account - successful
[2]Created new account - 8a85860c6a25f24b016a26825f6445c7
Changed Status to approved - successful
Delete account - successful
[3]Created new account - 8a8587de6a25966b016a268274d463ce
Changed Status to approved - successful
Delete account - successful
[4]Created new account - 8a8586e66a25df09016a268278604f7d
Changed Status to approved - successful
Delete account - successful
[5]Created new account - 8a8586e66a25df09016a2682a2544f87
Changed Status to approved - successful
Delete account - successful
[6]Created new account - 8a8587856a25d48e016a2682a8a47a56
Changed Status to approved - successful
Delete account - successful
[7]Created new account - 8a8586e66a25df09016a2682a9c84f8c
Changed Status to approved - successful
Delete account - suc

In [43]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
# Step 1 -------------------- Create the account
r = POST('{{env3}}/deposits', headers=HEADERS, params=PARAMS, body="JSONBODY4")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
201
Now the JSON:
{"encodedKey":"8a8586e46a235584016a24e001b6193e","creationDate":"2019-04-16T07:41:12+01:00","lastModifiedDate":"2019-04-16T07:41:12+01:00","id":"BGWV647","name":"MKCurTest1","accountHolderType":"CLIENT","accountHolderKey":"8a85879169f1de4c0169f289795519da","accountState":"PENDING_APPROVAL","productTypeKey":"8a8586e46a235584016a24695e6214bd","accountType":"CURRENT_ACCOUNT","currencyCode":"GBP","assignedBranchKey":"8a85879169f1de4c0169f289795519d8","internalControls":{},"overdraftSettings":{"allowOverdraft":true,"overdraftLimit":0},"interestSettings":{"interestPaymentSettings":{"interestPaymentDates":[]}},"overdraftInterestSettings":{"interestRateSettings":{"encodedKey":"8a8586e46a235584016a24e001b6193f","interestRate":2.00000000000000000000,"interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED","interestRateSource":"FIXED_INTEREST_RATE"}},"balances":{"overdraftAmount":0,"technicalOverdraftA

## More information about the problem
It is happending when trying to create clients

In [50]:
%%writefile JSONBODY
{
"firstName": "Mambu",
"lastName": "Test",
"preferredLanguage": "ENGLISH",
"addresses": [
    {
      "country": "UK",
      "city": "Liverpool"
    }
],
"notes": "Some Notes on this person",
"gender": "MALE"   
}
  

    

Overwriting JSONBODY


In [65]:
def testScenario4():
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env3}}/clients', headers=HEADERS, params=PARAMS, body="JSONBODY")
    # Remember the ID of the customer created we will use it below 
    newCustomerID = r.json()['id']
    try:
        if r.status_code != 201:
            print("Failed to create client")
        else:
            print("Created new client - {0}".format(newCustomerID))
            r = GET("{{env3}}/clients/" + newCustomerID, headers=HEADERS, params=PARAMS)
            if r.status_code != 200:
                print("Failed to GET client")
            else:
                print("GET client - success")
            r = DELETE("{{env3}}/clients/" + newCustomerID, headers=HEADERS, params=PARAMS)
            if r.status_code != 204:
                print("Failed to delete client")
            else:
                print("Delete client - successful")
    except:
        print("Failed to create client")

In [66]:
for i in range(10):
    print("[{0}]".format(i), end='')
    testScenario4()


[0]Created new client - 885640553
GET client - success
Delete client - successful
[1]Created new client - 121958792
GET client - success
Delete client - successful
[2]Created new client - 113206579
GET client - success
Delete client - successful
[3]Created new client - 846451860
GET client - success
Delete client - successful
[4]Created new client - 353562412
GET client - success
Delete client - successful
[5]Created new client - 133035919
GET client - success
Delete client - successful
[6]Created new client - 831842200
GET client - success
Delete client - successful
[7]Created new client - 211208473
GET client - success
Delete client - successful
[8]Created new client - 633899996
GET client - success
Delete client - successful
[9]Created new client - 802640797
GET client - success
Delete client - successful


In [57]:
newCustomerID = '900825451'

In [58]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
PARAMS = {}
r = DELETE("{{env3}}/clients/" + newCustomerID, headers=HEADERS, params=PARAMS)
PRINT(r)

API status: 404
JSON Response:
{
    "errors": [
        {
            "errorCode": 301,
            "errorReason": "INVALID_CLIENT_ID"
        }
    ]
}
